In [13]:
import os
import shutil
import random
import xml.etree.ElementTree as ET
from pathlib import Path

# ===============================
# 🧱 配置参数区域
# ===============================
DATASET_URL = "andrewmvd/face-mask-detection"  # kaggle 数据集名称
DATASET_DIR = "dataset"        # 解压数据集的路径
ANNOT_DIR = os.path.join(DATASET_DIR, "annotations")
IMAGES_DIR = os.path.join(DATASET_DIR, "images")

OUTPUT_DIR = "dataset/output"
TRAIN_DIR = os.path.join(OUTPUT_DIR, "train")
TEST_DIR = os.path.join(OUTPUT_DIR, "test")
YAML_PATH = "voc.yaml"

CLASS_NAMES = ["with_mask", "without_mask"]
TRAIN_RATIO = 0.8

# ===============================
# 📥 下载并解压数据集
# ===============================
def download_dataset(dataset_url, output_dir):
    try:
        import kagglehub
    except ImportError:
        print("❌ 请先安装 kagglehub：pip install kagglehub")
        return

    if os.path.exists(output_dir):
        print(f"✅ 数据集已存在：{os.path.abspath(output_dir)}")
        return

    print("🚀 正在下载数据集中...")
    downloaded_path = kagglehub.dataset_download(dataset_url)
    shutil.copytree(downloaded_path, output_dir)
    shutil.rmtree(downloaded_path)
    print(f"✅ 数据集下载完成：{os.path.abspath(output_dir)}")

# ===============================
# 🔁 VOC → YOLO 格式转换函数
# ===============================
def convert_voc_to_yolo(xml_file, yolo_save_dir, class_names):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    img_width = int(root.find("size/width").text)
    img_height = int(root.find("size/height").text)
    label_lines = []

    for obj in root.findall("object"):
        cls_name = obj.find("name").text
        if cls_name not in class_names:
            continue
        cls_id = class_names.index(cls_name)

        bbox = obj.find("bndbox")
        xmin = int(bbox.find("xmin").text)
        ymin = int(bbox.find("ymin").text)
        xmax = int(bbox.find("xmax").text)
        ymax = int(bbox.find("ymax").text)

        # 转换为归一化中心坐标 + 宽高
        x_center = ((xmin + xmax) / 2) / img_width
        y_center = ((ymin + ymax) / 2) / img_height
        width = (xmax - xmin) / img_width
        height = (ymax - ymin) / img_height

        label_lines.append(f"{cls_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

    # 保存为 .txt 文件
    txt_file = Path(yolo_save_dir) / (Path(xml_file).stem + ".txt")
    with open(txt_file, "w") as f:
        f.write("\n".join(label_lines))

# ===============================
# 🔀 划分数据集并执行转换
# ===============================
def prepare_dataset(train_ratio):
    # 创建输出文件夹
    os.makedirs(TRAIN_DIR, exist_ok=True)
    os.makedirs(TEST_DIR, exist_ok=True)

    # 加载所有标注文件和对应图像
    xml_files = sorted(Path(ANNOT_DIR).glob("*.xml"))
    img_files = [Path(IMAGES_DIR) / (xml.stem + ".png") for xml in xml_files]

    # 验证文件是否存在
    for img in img_files:
        if not img.exists():
            raise FileNotFoundError(f"未找到图像文件：{img}")

    # 打乱并划分数据集
    data = list(zip(xml_files, img_files))
    random.shuffle(data)
    split_idx = int(len(data) * train_ratio)
    train_data, test_data = data[:split_idx], data[split_idx:]

    def process(data_list, target_dir):
        for xml_path, img_path in data_list:
            shutil.copy(xml_path, target_dir)
            shutil.copy(img_path, target_dir)
            convert_voc_to_yolo(xml_path, target_dir, CLASS_NAMES)

    process(train_data, TRAIN_DIR)
    process(test_data, TEST_DIR)

    print(f"✅ 数据集划分完成，训练集数量：{len(train_data)}，测试集数量：{len(test_data)}")

# ===============================
# 📝 生成 YOLO YAML 配置文件
# ===============================
def create_yaml(path, train_dir, val_dir, class_names):
    with open(path, "w") as f:
        f.write(f"train: {os.path.abspath(train_dir)}\n")
        f.write(f"val: {os.path.abspath(val_dir)}\n\n")
        f.write(f"nc: {len(class_names)}\n")
        f.write(f"names: {class_names}\n")
    print(f"📄 已生成配置文件：{path}")

# ===============================
# 🚀 主执行入口
# ===============================

download_dataset(DATASET_URL, DATASET_DIR)
prepare_dataset(TRAIN_RATIO)
create_yaml(YAML_PATH, TRAIN_DIR, TEST_DIR, CLASS_NAMES)


✅ 数据集已存在：/content/dataset
✅ 数据集划分完成，训练集数量：682，测试集数量：171
📄 已生成配置文件：voc.yaml


In [ ]:
!pip install ultralytics
!nvidia-smi
from ultralytics import YOLO
device = 'cuda'  # 使用GPU训练,可选cuda或cpu

model = YOLO("baseModel/yolov8n.pt")  # 使用预训练模型
model.train(
    data="voc.yaml",
    device=0 if device == "cuda" else "cpu",
    epochs=40,
    batch=64,
    imgsz=640,
    optimizer="AdamW",
    augment=True,
    lr0=0.001,               # 适当提高初始学习率
    lrf=0.01,                # 添加余弦退火最终学习率
    amp=True,               # 保持混合精度训练
    pretrained=True,        # 确保使用预训练权重
    save=True,  # 保存模型
    exist_ok=True,
    )  # 训练模型

Sat Apr 19 09:56:01 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   69C    P0             31W /   70W |    9700MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

train: Scanning /content/dataset/output/train... 682 images, 17 backgrounds, 0 corrupt: 100%|██████████| 682/682 [00:00<00:00, 686.75it/s]

train: New cache created: /content/dataset/output/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1029.1±642.5 MB/s, size: 340.0 KB)


val: Scanning /content/dataset/output/test... 171 images, 4 backgrounds, 0 corrupt: 100%|██████████| 171/171 [00:00<00:00, 423.13it/s]


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      26/40      9.52G      1.015     0.5692     0.9409        403        640: 100%|██████████| 11/11 [00:08<00:00,  1.27it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:03<00:00,  1.58s/it]

                   all        171        805      0.863      0.795      0.865      0.551



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      27/40      7.54G      1.003     0.5733     0.9607        531        640:   9%|▉         | 1/11 [00:00<00:07,  1.27it/s]